In [12]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
voyage_code = 'MIR01'
df = df[df['voyage_code']==voyage_code]
sl = list(df['start_loc'])[0]
ll = list(df['last_loc'])[0]
dvb = list(df['date_voy_begin'])[0]
dve = list(df['date_voy_end'])[0]
# df= df[['n', 'voyage_code', 'last_name', 'first_name', 'f_n', 'remarks', 'age', 'wage',
#        'emb_date', 'disemb_date', 'disemb_loc', 'steps']].drop_duplicates()
# df.rename(columns={'last_name': 'Last Name', 'first_name': 'First Name', 'f_n': 'Function', 'remarks': 'Remarks', 'age': 'Age', 'wage': 'Wage',
#        'emb_date': 'Emb_date', 'disemb_date': 'Disemb_date', 'disemb_loc': 'Disemb_loc', 'voyage_code': 'Voyage Code'}, inplace=True)
# df_joined = pd.read_csv('_SELECT_r_p_v_vd_FROM_record_r_JOIN_person_p_ON_r_person_index_p_202408261231.csv').drop('Unnamed: 0', axis=1)
# df_joined = df_joined[(df_joined['last_loc'] == ll) & (df_joined['start_loc'] == sl) & (df_joined['date_voy_begin'] == dvb) & (df_joined['date_voy_end'] == dve)]
# for col in df.select_dtypes(include=['object']):
#     df[col] = df[col].str.strip()
# df.replace('[nan]', np.nan, inplace=True)
# df.replace('<Disemb_Loc>', np.nan, inplace=True)

<ipython-input-12-80263e491096>:5: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)


In [14]:
df['last_loc']

170797    Hélène → Lorient
170798    Hélène → Lorient
170799    Hélène → Lorient
170800    Hélène → Lorient
170801    Hélène → Lorient
                ...       
208584    Hélène → Lorient
208585    Hélène → Lorient
208586    Hélène → Lorient
208587    Hélène → Lorient
208588    Hélène → Lorient
Name: last_loc, Length: 121, dtype: object

In [4]:
df

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,steps
170797,1.0,MIR01,DORVO,Claude Thomas,16-62,a fait la campagne --- habitué à Lorient,50.0,200,NaN,NaN,<Disemb_Loc>,NaN
170798,117.0,MIR01,CHAPELAIN,Étienne,NaN,embarqué en rivière de Chandernagor le \n 22/0...,NaN,NaN,NaN,NaN,NaN,NaN
170799,118.0,MIR01,BENARD,René,NaN,embarqué en rivière de Chandernagor le \n 22/0...,NaN,NaN,NaN,NaN,NaN,NaN
170800,78.0,MIR01,LALANDE,Jean,NaN,a fait la campagne,33.0,40,NaN,NaN,<Disemb_Loc>,NaN
170801,28.0,MIR01,DARGENT,Étienne Auguste,NaN,a fait la campagne --- 1er voyage,11.0,10,NaN,NaN,<Disemb_Loc>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
208584,103.0,MIR01,LAGUNE,Jean,NaN,remplacement à Chandernagor du 27/12/1786 \n d...,NaN,26,NaN,NaN,NaN,NaN
208585,101.0,MIR01,GUERIN,Pierre,NaN,remplacement à Chandernagor du 18/12/1786 \n d...,NaN,26,NaN,NaN,NaN,NaN
208586,94.0,MIR01,BORDIER,Gilles,NaN,remplacement à Chandernagor du 29/09/1786 \n d...,NaN,22,NaN,NaN,NaN,NaN
208587,95.0,MIR01,LE CAM,Corentin,NaN,remplacement à Chandernagor du 29/09/1786 \n d...,NaN,10,NaN,NaN,NaN,NaN


In [5]:
def extract_date(text):
    # Helper function to extract date in the format dd/mm/yyyy
    match = re.search(r'(\d{2}/\d{2}/\d{4})', text)
    if match:
        return pd.to_datetime(match.group(1), dayfirst=True)
    return None

def extract_disembark_loc(text):
    # Helper function to extract the disembark location from text and clean it up
    match = re.search(r'débarqué[es]? à ([A-Za-zÀ-ÿ\'-]+(?: [A-Za-zÀ-ÿ\'-]+)*)', text, flags=re.IGNORECASE)
    if match:
        place_name = match.group(1)
        # Remove trailing non-place words like "le", "la", "du", etc.
        place_name = re.sub(r'\b(?:le|la|du|en|aux)\b.*$', '', place_name, flags=re.IGNORECASE).strip()
        return place_name
    return None

def split_remarks(remarks):
    # Split remarks based on 'rembarqué' keyword to handle each segment separately
    parts = re.split(r'(rembarqué[es]?)', remarks, flags=re.IGNORECASE)
    segments = [parts[0]]  # First part before 'rembarqué'
    
    for i in range(1, len(parts) - 1, 2):
        segments.append(parts[i] + parts[i + 1])  # Combine 'rembarqué' with the following text
    
    return segments

def process_rembarque(df):
    new_rows = []

    # Filter rows that contain the "rembarqué" keyword in the Remarks column
    rembarque_rows = df[df['Remarks'].str.contains('rembarqué', na=False, case=False)]

    for index, row in rembarque_rows.iterrows():
        segments = split_remarks(row['Remarks'])

        # Process the first segment (before the first rembarqué)
        first_segment = segments[0]
        first_embark_date = row['Emb_date']  # Use original embark date
        first_disembark_date = extract_date(re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE) else None
        first_disembark_loc = extract_disembark_loc(first_segment)  # Extract disembark location

        original_row = row.copy()
        original_row['Emb_date'] = first_embark_date  # Keep original embark date
        original_row['Disemb_date'] = first_disembark_date if first_disembark_date else row['Disemb_date']
        original_row['Disemb_loc'] = first_disembark_loc if first_disembark_loc else row['Disemb_loc']
        original_row['Remarks'] = first_segment.strip()
        new_rows.append(original_row)

        previous_disembark_date = first_disembark_date

        # Process subsequent segments starting with "rembarqué"
        for i, segment in enumerate(segments[1:], start=1):
            # Handle "rembarqué le dit jour et an"
            if 'rembarqué le dit jour et an' in segment.lower():
                embark_date = previous_disembark_date
            else:
                embark_date = extract_date(re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE) else None

            disembark_date = extract_date(re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE) else None
            disembark_loc = extract_disembark_loc(segment)  # Extract disembark location

            new_row = row.copy()
            new_row['Emb_date'] = embark_date
            new_row['Disemb_date'] = disembark_date if disembark_date else row['Disemb_date']
            
            # Only the last segment should keep the original Disemb_loc; others should extract it
            if i == len(segments) - 1:
                new_row['Disemb_loc'] = row['Disemb_loc']
            else:
                new_row['Disemb_loc'] = disembark_loc if disembark_loc else row['Disemb_loc']

            new_row['Remarks'] = segment.strip()
            new_rows.append(new_row)

            # Update the previous disembark date
            previous_disembark_date = disembark_date if disembark_date else previous_disembark_date

    # Append the processed rows back to the original dataframe
    non_rembarque_rows = df[~df.index.isin(rembarque_rows.index)]
    new_df = pd.concat([non_rembarque_rows, pd.DataFrame(new_rows)], ignore_index=True)

    # Sort the dataframe by Last Name and Embark Date
    new_df = new_df.sort_values(by=['Last Name', 'Emb_date'])

    return new_df

def replace_nan_array(val):
    if isinstance(val, np.ndarray):
        if val.size == 1 and np.isnan(val[0]):
            return np.nan
    return val

def match_or_nan(a, b):
    # Function to check if two values are equal or both are NaN
    return (a == b) | (pd.isna(a) & pd.isna(b))

def check_issues(row):
    issues = []
    if not pd.isna(row['Emb_date']) and not pd.Series(row['Emb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('emb_date')
    if pd.isna(row['Emb_date']):
        issues.append('emb_date')
    if not pd.isna(row['Disemb_date']) and not pd.Series(row['Disemb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('disemb_date')
    if pd.isna(row['Disemb_date']):
        issues.append('Disemb_date')
    if pd.isna(row['Emb_loc']):
        issues.append('emb_loc')
    if pd.isna(row['Disemb_loc']):
        issues.append('disemb_loc')
    return issues


def convert_to_ddmmyyyy(date_value):
    if pd.isna(date_value) or str(date_value).lower() in ['nan', '[nan]']:
        return np.nan
    try:
        # Parse the date and reformat to dd/mm/yyyy
        return pd.to_datetime(date_value).strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        return date_value


def extract_details(text):
    # Patterns for embarkation
    embark_patterns = {
        'embarqué': {                                                                                           
            'date': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:(?!\b(?:débarqué|noyé(?:e|es|s)?|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*le (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:clandestinement à|furtivement à|furtivement au|au|à|en supplément à) ([\w\s\'-îÎ]+?)(?=\s+\b(?:départ|mort(?:e|es|s)?|débarqué(?:e|es|s)?|déserté(?:e|es|s)?|passé(?:e|es|s)?|resté(?:e|es|s)?|tombé(?:e|es|s)?|repassé(?:e|es|s)?)\b| le \d{2}/\d{2}/\d{4}|,| ---|\n|$| a servi à bord)'
        },
        'a fait la': {
            'date': r'remplacement du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de ([\w\s\'-îÎ]+?) à ([\w\s\'-îÎ]+)'
        },
        'supplément': {
            'date': r'\bsupplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bsupplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'complément': {
            'date': r'\bcomplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bcomplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'remplacement': {
            'date': r'\bremplacement (?:au|à)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+?(?: le| du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bremplacement (?:au|à)? ([\w\s\'-îÎ]+?)(?: en| le| du|\n|,| ---|$)'
        },
        'trouvé': {
            'date': r'\btrouvé(?:e|es|s)? caché(?:e|es|s)? à bord(?:(?!\b(?:débarqué(?:e|es|s)?|déserté(?:e|es|s)?|mort(?:e|es|s)?|passé(?:e|es|s)?)\b)[^\n])*? le (\d{2}/\d{2}/\d{4})',
            'location': r'\btrouvé(?:e|es|s)? caché(?:e|es|s)? à bord (?:après le départ de|le \d{2}/\d{2}/\d{4})\s+((?!\b(?:débarqué(?:e|es|s)?|sert|déserté(?:e|es|s)?|mort(?:e|es|s)?|passé(?:e|es|s)?)\b)[\w\s\'-îÎ]+?)(?=\s+\b(?:le\s+\d{2}/\d{2}/\d{4}|---|,|\n|$|\b(?:débarqué(?:e|es|s)?|sert|déserté(?:e|es|s)?|mort(?:e|es|s)?|passé(?:e|es|s)?)\b))'
        }, 
        'soldat': {
            'date': r'\bsoldat(?:e|es|s)? à bord le\s*(\d{2}/\d{2}/\d{4})(?:\s|,| ---|$)',
            'location': r''
        },
        'né': {
            'date': r'\bné(?:e|es|s)? (?:à bord|en mer) le (\d{2}/\d{2}/\d{4})',
            'location': r''
        }
    }

    # Patterns for disembarkation
    disembark_patterns = {
        'débarqué': {
            'date': r'\bdébarqué(?:e|es|s)? (?:à l\'hôpital de |à |au |furtivement à |malade après être tombé du haut mal à |au désarmement |désarmement à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) )?[^\d\n\-,]*?le (\d{2}/\d{2}/\d{4})(?=(\s*---|\n|$|\s*passé(?:s|e|es)?|\s*échappé(?:e|s|es)?|\s*mort(?:e|s|es)?|\s*noyé(?:e|s|es)?))',
            'location': r'\bdébarqué(?:e|es|s)? (?:à l\'hôpital de |à |au |furtivement à |malade après être tombé du haut mal à |au désarmement |désarmement à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) )?([^\d\n\-,]+?)(?=( ---|---|\s+le\s+\d{2}/\d{2}/\d{4}|le\s+\d{2}/\d{2}/\d{4}|\n|$|\s*passé(?:s|e|es)?|échappé(?:e|s|es)?| échappé(?:e|s|es)?|mort(?:e|s|es)?| mort(?:e|s|es)?|noyé(?:e|s|es)?| noyé(?:e|s|es)?))'
        },
        'déserté': {
            'date': r'\bdéserté(?:e|es|s)? (?:à [\w\s\'-îÎ]+|sur le vaisseau de côte le [\w\s\'-îÎ]+|en [\w\s\'-îÎ]+|au départ à [\w\s\'-îÎ]+) le (\d{2}/\d{2}/\d{4})',
            'location': r'\bdéserté(?:e|es|s)? (?:à|en|au départ de|au départ à) ([\w\s\'-îÎ]+?)(?: le|\n|,| ---|$| au départ)'
        },
        'mort en mer': {
            'date': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer |noyé en [\w\s\'-îÎ]+ |du [\w\s\'-îÎ]+ en [\w\s\'-îÎ]+ |à l\'hôpital de [\w\s\'-îÎ]+ |à la ration |à [\w\s\'-îÎ]+ |tombé à la mer et mort noyé |dans le Gange |au [\w\s\'-îÎ]+ )?(?:le|le )(\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer|à l\'hôpital (?:du|de)|à la ration|à|au) ([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4}| le\d{2}/\d{2}/\d{4}| ---| \n)'
        },
        'a fait la': {
            'date': r'levé(?:e|es|s)? du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de [\w\s\'-îÎ]+? à ([\w\s\'-îÎ]+?)(?=\s*(?:du|le) \d{2}/\d{2}/\d{4}|\s*---|,|en|\n|$)'
        },
        'passé': {
            'date': r'\bpassé(?:e|es|s)? sur la [\w\s\'-îÎ]+ en rade de [\w\s\'-îÎ]+ le (\d{2}/\d{2}/\d{4})',
            'location': r'\bpassé(?:e|es|s)? sur la [\w\s\'-îÎ]+ en rade de ([\w\s\'-îÎ]+)(?=\s+(?:le\s+\d{2}/\d{2}/\d{4}|,| ---|\n|$))'
        },
        'resté': {
            'date': r'\bresté(?:e|es|s)?(?:\s+(?:à terre à|à terre|à l\'hôpital à|à l\'hôpital de|malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|malade à|malade)?(?: [\w\s\'-îÎ]+?)?)? le\s*(\d{2}/\d{2}/\d{4})(?:\b.*?| rejoint|\n|,| ---|$)',
            'location': r'\bresté(?:e|es|s)? (?:à terre à|à terre|à l\'hôpital à|à l\'hôpital de|malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|malade à) ([\w\s\'-îÎ]+?)(?: en supplément à| le| rejoint|\n|,| ---|$| au départ)'
        }
    }

    # Initialize results
    embark_location = pd.NA
    embark_date = pd.NA
    disembark_location = pd.NA
    disembark_date = pd.NA

    # Initialize match variables
    embark_date_match = None
    embark_location_match = None
    disembark_date_match = None
    disembark_location_match = None

    # Determine which embark patterns to use
    if 'embarqué' in text.lower() or 'rembarqué' in text.lower():
        embark_date_match = re.search(embark_patterns['embarqué']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embarqué']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        embark_date_match = re.search(embark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'supplément' in text.lower():
        embark_date_match = re.search(embark_patterns['supplément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['supplément']['location'], text, flags=re.IGNORECASE)
    elif 'remplacement' in text.lower():
        embark_date_match = re.search(embark_patterns['remplacement']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['remplacement']['location'], text, flags=re.IGNORECASE)
    elif 'complément' in text.lower():
        embark_date_match = re.search(embark_patterns['complément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['complément']['location'], text, flags=re.IGNORECASE)        
    elif 'trouvé' in text.lower():
        embark_date_match = re.search(embark_patterns['trouvé']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['trouvé']['location'], text, flags=re.IGNORECASE)        
    elif 'soldat à bord le' in text.lower():
        embark_date_match = re.search(embark_patterns['soldat']['date'], text, flags=re.IGNORECASE)
    elif re.search(r'\bné(?:e|es|s)?\b', text.lower()):
        embark_date_match = re.search(embark_patterns['né']['date'], text, flags=re.IGNORECASE)

    # Extract embark information
    if embark_date_match:
        embark_date = embark_date_match.group(1)
    if embark_location_match:
        embark_location = embark_location_match.group(1)

    # Determine which disembark patterns to use
    if 'débarqué' in text.lower():
        disembark_date_match = re.search(disembark_patterns['débarqué']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['débarqué']['location'], text, flags=re.IGNORECASE)
    elif 'déserté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['déserté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['déserté']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        disembark_date_match = re.search(disembark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'resté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['resté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['resté']['location'], text, flags=re.IGNORECASE)
    elif 'passé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['passé']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['passé']['location'], text, flags=re.IGNORECASE)
    elif 'mort' in text.lower() or 'noyé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['mort en mer']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['mort en mer']['location'], text, flags=re.IGNORECASE)

    # Extract disembark information
    if disembark_date_match:
        disembark_date = disembark_date_match.group(1)
    if disembark_location_match:
        disembark_location = disembark_location_match.group(1)
#         if len(disembark_location_match.groups()) > 1 and disembark_location_match.group(2):
#             disembark_location += f", {disembark_location_match.group(2)}"
    if 'mort à bord' in text.lower() or 'morte à bord' in text.lower() or 'morts à bord' in text.lower() or 'mortes à bord' in text.lower():
        disembark_location = 'on board'
    elif 'mort en mer' in text.lower() or 'morte en mer' in text.lower() or 'morts en mer' in text.lower() or 'mortes en mer' in text.lower():
        disembark_location = 'on board'
    if 'née en mer' in text.lower() or 'né en mer' in text.lower() or 'nées en mer' in text.lower() or 'nés en mer' in text.lower():
        embark_location = 'on board'
    elif 'née à bord' in text.lower() or 'né à bord' in text.lower() or 'nées à bord' in text.lower() or 'nés à bord' in text.lower():
        embark_location = 'on board'
    
            

    return embark_location, embark_date, disembark_location, disembark_date

def fill_emb_loc_for_rembarque(df):
    # Sort the dataframe by Last Name, Emb_date, and Disemb_date
    df_sorted = df.sort_values(by=['Last Name', 'Emb_date', 'Disemb_date']).reset_index(drop=True)
    
    # Iterate through the sorted dataframe
    for i in range(1, len(df_sorted)):
        current_row = df_sorted.iloc[i]
        previous_row = df_sorted.iloc[i - 1]
        
        # Check if Remarks is a string and contains the keyword "rembarqué"
        if pd.isna(current_row['Emb_loc']) and isinstance(current_row['Remarks'], str) and 'rembarqué' in current_row['Remarks'].lower():
            # Set the current row's Emb_loc to the previous row's Disemb_loc
            df_sorted.at[i, 'Emb_loc'] = previous_row['Disemb_loc']
    
    return df_sorted

In [6]:
# split by re-embark
df = process_rembarque(df)

In [7]:
# Extracting info from Remarks

df['Emb_loc'] = None
for index, row in df.iterrows():
    remark = re.sub(r'\s+', ' ', str(row['Remarks']).replace('\n', ' '))
    
    emb_date = row['Emb_date']
    disemb_date = row['Disemb_date']
    disemb_loc = row['Disemb_loc']
    embark_location, embark_date, disembark_location, disembark_date = extract_details(remark)
    
    df.at[index, 'Emb_loc'] = embark_location
    if pd.notna(disembark_date):
        df.at[index, 'Disemb_date'] = disembark_date
    if pd.notna(embark_date):
        df.at[index, 'Emb_date'] = embark_date
    if pd.notna(disembark_location):
        df.at[index, 'Disemb_loc'] = disembark_location

# Apparently, fill emb_loc for re-embark
df = fill_emb_loc_for_rembarque(df)

In [8]:
steps = df['steps'][0]
stops = []
if isinstance(steps, str):
    clean = steps.replace(",", "-")
    temp = re.split(r'(?<=\w)-(?=\s)|(?<=\s)-(?=\w)|(?<=\s)-(?=\s)', clean)
    for j in temp:
        stops.append(j.replace("?", "").strip())
stops

[]

In [21]:
# replacing "armement" and "désarmement"
des = stops[-1]
arm = stops[0]
for index, row in df.iterrows():
    if pd.notna(row['Remarks']):
        if isinstance(row['Disemb_loc'], str) and ('désarmement' in row['Disemb_loc'].lower() or row['Disemb_loc'].lower()=='désarmement'):
            df.at[index, 'Disemb_loc'] = des
                
            if pd.isna(row['Disemb_date']):
                df.at[index, 'Disemb_date'] = dve

        if isinstance(row['Emb_loc'], str) and 'armement' in row['Emb_loc'].lower():
            df.at[index, 'Emb_loc'] = arm
                
            if pd.isna(row['Emb_date']):
                df.at[index, 'Emb_date'] = dvb
        
        if 'a fait la campagne' in row['Remarks'].lower():
            if pd.isna(row['Emb_date']):
                df.at[index, 'Emb_date'] = dvb
            if pd.isna(row['Disemb_date']):
                df.at[index, 'Disemb_date'] = dve
            if pd.isna(row['Emb_loc']):
                df.at[index, 'Emb_loc'] = arm
            if pd.isna(row['Disemb_loc']):
                df.at[index, 'Disemb_loc'] = des

In [22]:
df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)

<ipython-input-17-28b650838df0>:120: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')


In [23]:
# Add emb_class and disemb_class

df['emb_class'], df['disemb_class'] = None, None
for index, row in df.iterrows():
    if not isinstance(row['Remarks'], str):
        continue
    if 'clandestin' in row['Remarks'] or 'caché' in row['Remarks'] or 'furtivement' in row['Remarks']:
        df.at[index, 'emb_class'] = 304
    elif 'rembarqué' in row['Remarks']:
        df.at[index, 'emb_class'] = 306
    elif 'fait la campagne' in row['Remarks'] or 'armement' in row['Remarks'] or 'embarqué' in row['Remarks']:
        if pd.notna(row['Emb_loc']):
            if row['Emb_loc'] == arm:
                df.at[index, 'emb_class'] = 301
            else:
                df.at[index, 'emb_class'] = 302
    elif 'remplacement' in row['Remarks'] or 'supplément' in row['Remarks']:
        df.at[index, 'emb_class'] = 302
    elif 'resté' in row['Remarks']:
        df.at[index, 'emb_class'] = 308
    elif 'renversement' in row['Remarks'] or 'vient' in row['Remarks']:
        df.at[index, 'emb_class'] = 303
    elif 'né' in row['Remarks']:
        df.at[index, 'emb_class'] = 305
    else:
        df.at[index, 'emb_class'] = 309
        
    if 'mort' in row['Remarks'] or 'noyé' in row['Remarks']:
        df.at[index, 'disemb_class'] = 305
    elif 'fait la campagne' in row['Remarks']:
        df.at[index, 'disemb_class'] = 301
    elif 'resté malade' in row['Remarks'] or 'resté à l\'hôpital' in row['Remarks'] or 'restée malade' in row['Remarks'] or 'restée à l\'hôpital' in row['Remarks']:
        df.at[index, 'disemb_class'] = 306
    elif 'débarqué' in row['Remarks']:
        if pd.notna(row['Disemb_loc']):
            if row['Disemb_loc'] == des:
                df.at[index, 'disemb_class'] = 301
            else:
                df.at[index, 'disemb_class'] = 302
    elif 'passé' in row['Remarks']:
        df.at[index, 'disemb_class'] = 303
    elif 'échappé' in row['Remarks'] or 'fugitif' in row['Remarks'] or 'déserté' in row['Remarks']:
        df.at[index, 'disemb_class'] = 304
    elif 'prise par le' in row['Remarks']:
        df.at[index, 'disemb_class'] = 307
    elif 'resté' in row['Remarks']:
        df.at[index, 'disemb_class'] = 308
    else:
        df.at[index, 'disemb_class'] = 309

In [24]:
# Well... filling based on re-embark???

reembark_rows = df[(df['Remarks'].str.contains('rembarqué', na=False, case=False)) & (df['Emb_loc'].isna())]

# Step 5: Process each row with the "rembarqué" keyword individually
for index, row in reembark_rows.iterrows():
    # Find rows with matching 'Last Name', 'First Name', and 'Function', handling NaN values
    matching_rows = df[
        match_or_nan(df['Last Name'], row['Last Name']) &
        match_or_nan(df['First Name'], row['First Name']) &
        match_or_nan(df['Function'], row['Function']) &
        (df['Last Name']!='Anonymous')
    ]

    # Sort the matching rows by Emb_date
    sorted_group = matching_rows.sort_values(by=['Emb_date', 'Disemb_date'])

    # Iterate through the sorted group to assign emb_loc based on previous Disemb_loc
    previous_disemb_loc = None
    for i, (sorted_index, sorted_row) in enumerate(sorted_group.iterrows()):
        if sorted_index == index:
            if previous_disemb_loc:
                df.at[sorted_index, 'Emb_loc'] = previous_disemb_loc

        # Update previous_disemb_loc for the next row in the sequence
        previous_disemb_loc = sorted_row['Disemb_loc']
        
for _, group in df.groupby(['Last Name', 'First Name', 'Function'], dropna=False):
    sorted_group = group.sort_values(by=['Emb_date', 'Disemb_date'])
    for i, (index, row) in enumerate(sorted_group.iterrows()):
        if pd.isna(row['Disemb_loc']) and i < len(sorted_group) - 1:
            if row['Last Name'] != 'Anonymous':
                next_row = sorted_group.iloc[i + 1]
                df.at[index, 'Disemb_loc'] = next_row['Emb_loc']

In [25]:
# Fill remaining from database
error_count = 0
for index, row in df.iterrows():
    try:
        if pd.notna(row['Remarks']):

            if row['Disemb_loc']=='ration':
                df.at[index, 'Disemb_loc'] = np.nan

    except:
        error_count += 1
#         print(row)
#         print('****************************************************************')
print(error_count)

0


In [26]:
df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Emb_loc'] = df['Emb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_loc'] = df['Disemb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
df.replace('[nan]', np.nan, inplace=True)

<ipython-input-17-28b650838df0>:120: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')


In [27]:
for index, row in df.iterrows():
    if isinstance(row['Remarks'], str) and ('mort' in row['Remarks'] or 'noyé' in row['Remarks']):
        if pd.notna(row['Disemb_date']):
            df.at[index, 'Disemb_loc'] = 'mort le '+row['Disemb_date']
            df.at[index, 'Disemb_date'] = np.nan
        else:
            df.at[index, 'Disemb_loc'] = 'mort en mer'
    if isinstance(row['Remarks'], str) and re.search(r'\bné(?:e|s|es)?\b', row['Remarks']):
        if pd.notna(row['Emb_date']):
            df.at[index, 'Emb_loc'] = 'né le '+row['Emb_date']
            df.at[index, 'Emb_date'] = np.nan
        else:
            df.at[index, 'Emb_loc'] = 'né en mer'
    if isinstance(row['Emb_loc'], str) and 'départ' in row['Emb_loc']:
        df.at[index, 'Emb_loc'] = np.nan
    if isinstance(row['Disemb_loc'], str) and 'départ' in row['Disemb_loc']:
        df.at[index, 'Disemb_loc'] = np.nan

In [28]:
df

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,steps,Emb_loc,emb_class,disemb_class
0,34.0,BEL03,ANGLES,Étienne,NaN,remplacement à St-louis de St-Domingue du \n 0...,NaN,60,01/03/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,St-louis de St-Domingue,302,None
1,41.0,BEL03,AUGER,Louis,NaN,remplacement à St-Louis de St-Domingue du \n 1...,NaN,30,19/10/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,St-Louis de St-Domingue,302,308
2,38.0,BEL03,AUGER,François,NaN,remplacement à St-Louis de St-Domingue du \n 1...,NaN,30,19/10/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,St-Louis de St-Domingue,302,308
3,29.0,BEL03,BEAL,Jean Pierre,NaN,complément à ? du 18/01/1781 \n déserté le 20/...,NaN,120,18/01/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,?,309,304
4,11.0,BEL03,BEDESQUE,Marc,NaN,a fait la campagne --- doit 72 £ pour frais de...,NaN,55,28/07/1780,NaN,Lorient,Lorient - Bordeaux - Rochefort - Saint-Domingu...,Lorient,301,301
5,8.0,BEL03,BERNARD,Jean,NaN,embarqué à l'armement \n déserté à Bordeaux le...,NaN,90,28/07/1780,23/08/1780,Bordeaux,Lorient - Bordeaux - Rochefort - Saint-Domingu...,Lorient,301,304
6,17.0,BEL03,BOZEC,François,NaN,embarqué à l'armement \n passé sur le Northumb...,NaN,36,28/07/1780,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,Lorient,301,303
7,7.0,BEL03,CALVÉ,Bertrand,NaN,embarqué à l'armement \n passé sur l'Actionnai...,NaN,80,28/07/1780,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,Lorient,301,303
8,36.0,BEL03,CAPIUS,François,NaN,remplacement à St-Louis de St-Domingue du \n 1...,NaN,40,19/10/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,St-Louis de St-Domingue,302,308
9,44.0,BEL03,CHALLES,NaN,NaN,remplacement à St-Louis de St-Domingue du \n 1...,NaN,sans solde,19/10/1781,NaN,NaN,Lorient - Bordeaux - Rochefort - Saint-Domingu...,St-Louis de St-Domingue,302,308


In [138]:
df[df['Last Name']=='DRIEN']

,n,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,Emb_loc,emb_class,disemb_class
82,233.0,DRIEN,Marin,NaN,embarqué clandestinement à Lorient \n a servi ...,NaN,NaN,NaN,29/12/1752,Chandernagor,Lorient,304,306


In [139]:
df.to_csv(voyage_code+'_filled.csv')

In [61]:
# df[df['issued'].apply(lambda x: len(x)>0)]

In [39]:
df

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,steps,Emb_loc,emb_class,disemb_class
0,18.0,CRX02,BARRONNEY,Benoît,270-279,embarqué à l'armement \n débarqué à ?,18.0,12,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
1,8.0,CRX02,BRIVAL,Jean Marie,853-2117,embarqué à l'armement \n débarqué à ?,24.0,24,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
2,21.0,CRX02,CARON,Gatien Martin,NaN,embarqué à l'armement \n débarqué à ? --- sans...,13.0,sans solde,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
3,2.0,CRX02,CHARLET,Jean,343-374,embarqué à l'armement \n débarqué à ? --- sans...,50.0,sans solde,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
4,15.0,CRX02,CHOUX,Thomas,NaN,embarqué à l'armement \n débarqué à ?,18.0,15,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
5,1.0,CRX02,DORVEAU,Claude,330-296,embarqué à l'armement \n débarqué à ?,40.0,150,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
6,13.0,CRX02,EVELIN,Jean,NaN,embarqué à l'armement \n débarqué à ?,20.0,18,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
7,4.0,CRX02,HAMELIN,Pierre,NaN,embarqué à l'armement \n débarqué à ?,20.0,27,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
8,19.0,CRX02,HUBERT,Claude Georges,NaN,embarqué à l'armement \n débarqué à ?,20.0,12,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
9,14.0,CRX02,JEGU,François,NaN,embarqué à l'armement \n débarqué à ?,20.0,15,14/03/1775,NaN,?,Lorient - Bengale ?,Lorient,301,302
